#### Basic LLMChain

In [5]:
from langchain import PromptTemplate, LLMChain

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [7]:
# Define the prompt template
prompt = PromptTemplate.from_template("What is the capital of {country}?")

In [8]:
# Initialize the LLM and chain
llm   = ChatOpenAI()

chain = LLMChain(llm   = llm, 
                 prompt= prompt)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_16148\2838644879.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm   = llm,


In [9]:
result = chain.invoke({"country": "France"})
print(result)  # Output should be "Paris"

{'country': 'France', 'text': 'The capital of France is Paris.'}


In [11]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [12]:
chain = LLMChain(llm=llm, prompt=prompt)

In [13]:
chain = LLMChain(llm=llm, prompt=prompt)

#### Simple Sequential Chain

In [16]:
llm_model = 'gpt-mini'

In [17]:
from langchain.chains import SimpleSequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [19]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [20]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [21]:
overall_simple_chain.run(product)

NameError: name 'product' is not defined

-----------------------
#### Sequential Chain

- A SequentialChain allows you to run multiple chains in sequence, where the output of one chain is used as the input to the next.
- ---------------------

In [22]:
from langchain.chains.sequential import SequentialChain

In [32]:
# Define the prompt templates
prompt1 = PromptTemplate.from_template("What is the capital of {country}?")
prompt2 = PromptTemplate.from_template("Describe the main attractions in {city}.")

In [33]:
# Initialize the language model
llm = ChatOpenAI()

In [34]:
# Create individual chains with explicit output
chain1 = LLMChain(llm       = llm, 
                  prompt    = prompt1, 
                  output_key="city")               # Sets "city" as output for chain1

chain2 = LLMChain(llm       = llm, 
                  prompt    = prompt2, 
                  output_key= "city_description")  # Sets "city_description" as output for chain2

In [35]:
# Set up the SequentialChain
sequential_chain = SequentialChain(
    chains           = [chain1, chain2],
    input_variables  = ["country"],          # Input to the first chain
    output_variables = ["city_description"]  # Output from the last chain
)

In [36]:
# Run the SequentialChain
result = sequential_chain.invoke({"country": "Bangladesh"})
print(result["city_description"]) 

Dhaka, the capital of Bangladesh, is a bustling city with a rich history and vibrant culture. Some of the main attractions in Dhaka include:

1. Lalbagh Fort: This 17th-century Mughal fort is a major historical site in Dhaka, featuring beautiful architecture and gardens.

2. Ahsan Manzil (Pink Palace): This stunning pink palace was once the residence of the Nawab of Dhaka and is now a museum showcasing the history and culture of Bangladesh.

3. National Museum: This museum houses a vast collection of artifacts and exhibits on the history, art, and culture of Bangladesh.

4. Liberation War Museum: This museum commemorates the Bangladesh Liberation War of 1971 and showcases the country's struggle for independence.

5. Dhakeshwari Temple: This ancient Hindu temple is one of the most important religious sites in Dhaka, attracting worshippers and visitors alike.

6. Star Mosque: This beautiful mosque is known for its intricate mosaics and tiles, making it a popular attraction for tourists.


Example

In [44]:
llm_model ='gpt-3.5-turbo'

In [45]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm       = llm, 
                     prompt    = first_prompt, 
                     output_key= "English_Review"
                    )


In [46]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 or 2 sentences:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm       = llm, 
                     prompt    = second_prompt, 
                     output_key= "summary"
                    )

In [47]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm       = llm, 
                       prompt    = third_prompt,
                       output_key= "language"
                      )


In [48]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm       = llm, 
                      prompt    = fourth_prompt,
                      output_key= "followup_message"
                     )

In [49]:
overall_chain = SequentialChain(
    chains          = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables= ["English_Review", "summary","followup_message"],
    verbose         = True
)

In [50]:
review = '''
বাংলাদেশ একটি সমৃদ্ধ ইতিহাস ও সংস্কৃতির দেশ। এই দেশে রয়েছে প্রাচীন স্থাপত্য, নদী, সবুজ প্রকৃতি এবং অসংখ্য জাতিগোষ্ঠীর বৈচিত্র্য। 
বাংলাদেশের মানুষ অতিথিপরায়ণ এবং তারা তাদের ঐতিহ্য ও সংস্কৃতিকে ভালোবাসে। প্রতি বছর এখানে বিভিন্ন ধরনের উৎসব পালন করা হয়, 
যেমন পহেলা বৈশাখ, ঈদ, দুর্গাপূজা, এবং বিজয় দিবস। এই উৎসবগুলোতে মানুষ একসঙ্গে আনন্দ করে, গান গায়, নাচে এবং খাবার খায়।

বাংলাদেশের অন্যতম বড় আকর্ষণ হচ্ছে সুন্দরবন, যা পৃথিবীর বৃহত্তম ম্যানগ্রোভ বন। এখানে রয়েল বেঙ্গল টাইগার, চিত্রা হরিণ, 
বানর এবং নানা ধরনের বন্যপ্রাণীর বসবাস। সুন্দরবনের প্রকৃতি অত্যন্ত সুন্দর এবং এটি প্রতিবছর হাজার হাজার পর্যটককে আকর্ষণ করে। 
তাছাড়া কক্সবাজার বিশ্বের দীর্ঘতম সমুদ্রসৈকত হিসাবে পরিচিত। সাদা বালুর সৈকত, নীল পানি, এবং সূর্যাস্তের দৃশ্য প্রতিটি দর্শকের মনে দাগ কাটে।

বাংলাদেশের মানুষের জীবনধারা খুবই সরল এবং প্রকৃতির সাথে গভীরভাবে সংযুক্ত। বেশিরভাগ মানুষ গ্রামাঞ্চলে বাস করে এবং তারা 
কৃষিকাজে ব্যস্ত থাকে। ধান, পাট, চা, এবং মাছ বাংলাদেশের প্রধান পণ্য। দেশটি ধীরে ধীরে শিল্প এবং প্রযুক্তিতে এগিয়ে যাচ্ছে এবং এশিয়ার 
একটি গুরুত্বপূর্ণ অর্থনৈতিক কেন্দ্রে পরিণত হচ্ছে।
'''

In [51]:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\nবাংলাদেশ একটি সমৃদ্ধ ইতিহাস ও সংস্কৃতির দেশ। এই দেশে রয়েছে প্রাচীন স্থাপত্য, নদী, সবুজ প্রকৃতি এবং অসংখ্য জাতিগোষ্ঠীর বৈচিত্র্য। \nবাংলাদেশের মানুষ অতিথিপরায়ণ এবং তারা তাদের ঐতিহ্য ও সংস্কৃতিকে ভালোবাসে। প্রতি বছর এখানে বিভিন্ন ধরনের উৎসব পালন করা হয়, \nযেমন পহেলা বৈশাখ, ঈদ, দুর্গাপূজা, এবং বিজয় দিবস। এই উৎসবগুলোতে মানুষ একসঙ্গে আনন্দ করে, গান গায়, নাচে এবং খাবার খায়।\n\nবাংলাদেশের অন্যতম বড় আকর্ষণ হচ্ছে সুন্দরবন, যা পৃথিবীর বৃহত্তম ম্যানগ্রোভ বন। এখানে রয়েল বেঙ্গল টাইগার, চিত্রা হরিণ, \nবানর এবং নানা ধরনের বন্যপ্রাণীর বসবাস। সুন্দরবনের প্রকৃতি অত্যন্ত সুন্দর এবং এটি প্রতিবছর হাজার হাজার পর্যটককে আকর্ষণ করে। \nতাছাড়া কক্সবাজার বিশ্বের দীর্ঘতম সমুদ্রসৈকত হিসাবে পরিচিত। সাদা বালুর সৈকত, নীল পানি, এবং সূর্যাস্তের দৃশ্য প্রতিটি দর্শকের মনে দাগ কাটে।\n\nবাংলাদেশের মানুষের জীবনধারা খুবই সরল এবং প্রকৃতির সাথে গভীরভাবে সংযুক্ত। বেশিরভাগ মানুষ গ্রামাঞ্চলে বাস করে এবং তারা \nকৃষিকাজে ব্যস্ত থাকে। ধান, পাট, চা, এবং মাছ বাংলাদেশের প্রধান পণ্য। দেশটি ধীরে ধীরে শিল্প এবং প্রযু